In [5]:
import os
from dotenv import load_dotenv
import requests

# .env에서 설정 읽기
load_dotenv()
TTS_API_KEY = os.getenv('TTS_API_KEY')
TTS_URL = os.getenv('TTS_URL')
STT_API_KEY = os.getenv('STT_API_KEY')
STT_URL = os.getenv('STT_URL')

# 변환할 한국어 텍스트
korean_text = "안녕하세요, 웹엠 파일 오디오 합성 예제입니다."

# 1. TTS (텍스트 → WebM 음성)
tts_headers = {
    'Content-Type': 'application/json',
    'Accept': 'audio/webm'  # WebM 오디오 파일 요청
}
tts_data = {
    'text': korean_text
}
tts_voice = 'ko-KR_JinV3Voice'
tts_endpoint = f"{TTS_URL}/v1/synthesize?voice={tts_voice}"

tts_response = requests.post(
    tts_endpoint,
    headers=tts_headers,
    json=tts_data,
    auth=('apikey', TTS_API_KEY)
)

if tts_response.status_code == 200:
    audio_file_path = "output_korean.webm"
    with open(audio_file_path, 'wb') as f:
        f.write(tts_response.content)
    print("TTS WebM 음성 파일 저장 완료:", audio_file_path)
else:
    print("TTS 오류:", tts_response.status_code, tts_response.text)
    exit()

# 2. STT (WebM 음성 → 텍스트)
stt_headers = {
    'Content-Type': 'audio/webm',
    'Accept': 'application/json'
}
stt_model = "ko-KR_BroadbandModel"
stt_endpoint = f"{STT_URL}/v1/recognize?model={stt_model}"

with open(audio_file_path, 'rb') as audio:
    stt_response = requests.post(
        stt_endpoint,
        headers=stt_headers,
        data=audio,
        auth=('apikey', STT_API_KEY)
    )

if stt_response.status_code == 200:
    result_json = stt_response.json()
    # 첫 번째 결과의 transcript 추출
    recognized = result_json.get('results', [])
    if recognized:
        stt_text = recognized[0]['alternatives'][0]['transcript']
        print("STT 인식 결과:", stt_text)
    else:
        print("STT 인식 결과가 없습니다.", result_json)
else:
    print("STT 오류:", stt_response.status_code, stt_response.text)


TTS WebM 음성 파일 저장 완료: output_korean.webm
STT 인식 결과: 안녕하세요 왜 빈 파일로 디오 합성 예정입니다 
